## Preprocessing данных для подготовки финального датасета

In [197]:
import os 
import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns

In [198]:
path = './_data.csv'
df = pd.read_csv(path)

In [199]:
# Исключение адресов вне Москвы и полных дубликатов (за исключением колонок 'ID  объявления', 'Unnamed: 0' и 'Ссылка на объявление').
df = df[df["Адрес"].str.contains("Москва")]
cols = df.columns.to_list()
cols = [i for i in cols if i not in ['ID  объявления', 'Unnamed: 0', 'Ссылка на объявление']]
df.drop_duplicates(subset=cols, inplace=True)
# Исключение колонок, которые были признаны подлежащими исключению по результатам EDA и с учётом дополнительного обсуждения внутри команды.
to_drop = ['Unnamed: 0', 'Тип', 'Адрес', 'Телефоны', 'Описание', 'Площадь комнат, м2', 'Балкон', 'Окна', 'Санузел', 
           'Дополнительно', 'Название ЖК', 'Парковка', 'Серия дома', 'Высота потолков, м', 'Мусоропровод', 'Ссылка на объявление', 'Лифт']
df.drop(columns=to_drop, inplace=True)

In [200]:
# Работа с колонкой "Количество комнат": получение значения общего числа комнат в допустимом формате.
df['rooms'] = df['Количество комнат'].apply(lambda x: int(x.split(',')[0]) if pd.notnull(x) else x)
df.drop(columns=['Количество комнат'], inplace=True)
# Ввиду небольшого числа пропусков, команда сочла допустимым использование метода Complete Case Analysis с удалением строк с отсутствующими значениями.
df.dropna(subset=['rooms'], inplace=True)

In [201]:
# Работа с колонкой "Метро": получение наименования станций. Ввиду ресурсных ограничений и непроработанности гипотез, информация о времени перемещения до станции метро была исключена. 
df['stations'] = df['Метро'].apply(lambda x: x.split(' ')[1] if pd.notnull(x) else x)
df.drop(columns=['Метро'], inplace=True)
# Ввиду небольшого числа пропусков и учитывая содержание соответствующих строк, команда сочла допустимым использование метода Complete Case Analysis с удалением строк с отсутствующими значениями.
# Более качественным и устойчивым к искажениям решением было бы кодирование непосредственно адресов квартир (подлежит дополнительному анализу). 
df.dropna(subset=['stations'], inplace=True)
# OHE колонки 'stations' выполнен в конце колонки.

In [202]:
# Работа с колонкой "Площадь, м2": получение значения общего числа комнат в допустимом формате.
df['area'] = df['Площадь, м2'].apply(lambda x: float(x.split('/')[0]))
df.drop(columns=['Площадь, м2'], inplace=True)
# Команда не пришла к однозначному выводу о необходимости устранения выбросов (квартиры большой метражности) ввиду рисков утраты ценной информации.

In [203]:
# Работа с колонкой "Дом": получение значений этажности домов, этажей расположения квартир в домах. Ввиду значительного числа пропусков, значения типов домов исключены из финального датасета.
df['house'] = df['Дом'].apply(lambda x: x.split(', '))
df['floors'] = df.loc[:,'house'].apply(lambda x: x[0])
df['apartment_floors'] = df.loc[:,'floors'].apply(lambda x: int(x.split('/')[0]))
df['total_floors'] = df.loc[:,'floors'].apply(lambda x: int(x.split('/')[1]))
total_floors = df['total_floors'].unique().tolist()
total_floors = sorted(total_floors)
total_floors[-1]
# Устранение строки с выбросом (опечаткой) по высоте дома.
df = df[df['total_floors'] != 116]
df.drop(columns=['floors', 'house', 'Дом'], inplace=True)

In [204]:
# Работа с колонкой "Цена": получение значений ежемесячной стоимости аренды в требуемом формате. Ввиду отсутствия качественных гипотез о стоимости коммунальных услуг, данный параметр не был сохранён в датасете. 
df['price'] = df['Цена'].apply(lambda x: float(x.split()[0]))
df.drop(columns=['Цена'], inplace=True)

In [205]:
# Работа с колонкой "Ремонт": принимая во внимание явную градацию видов ремонтов по стоимости, принято решение заменить значения по шкале от 0 (Без ремонта) до 3 (Дизайнерский). 
# Команда исходила из предположения, что отсутствующие значения, вероятно, соответствуют квартирам, в которых не был произведён ремонт. 
df['Ремонт'] = df['Ремонт'].fillna(0)
dict_repairs = {0: 0, 'Без ремонта': 0, 'Косметический': 1, 'Евроремонт': 2,  'Дизайнерский': 3}
df['repairs'] = df['Ремонт'].apply(lambda x: dict_repairs[x])
df.drop(columns='Ремонт', inplace=True)

In [206]:
# Работа с колонкой "Можно с детьми/животными": создание двух колонок (children, animals). 
# Данное решение принято исходя из логики эксплицитного указания в объявлениях на разрешение проживания детей или животных в квартире (презюмируется запрет). 
# Соответственно, отсутствующие значения интерпретированы как запрет.
df['Можно с детьми/животными'].head(15)
df['temp_children_animals'] = df['Можно с детьми/животными'].apply(lambda x: x.split(', ') if pd.notnull(x) else [0])
df['children'] = df['temp_children_animals'].apply(lambda x: 1 if 'Можно с детьми' in x else 0)
df['animals'] = df['temp_children_animals'].apply(lambda x: 1 if 'Можно с животными' in x else 0)
df.drop(columns=['Можно с детьми/животными', 'temp_children_animals'], inplace=True)

In [207]:
# Замена оригинальных названий колонок на латиницу.
df.rename(columns={"ID  объявления": "ID"}, inplace=True)
# Выполнение one-hot encoding для колонки stations.
dummy_stations = pd.get_dummies(df['stations'], dtype=int)
transliteration_dict = {'А': 'A',
                        'Б': 'B',
                        'В': 'V',
                        'Г': 'G',
                        'Д': 'D',
                        'Е': 'E',
                        'Ё': 'E',
                        'Ж': 'ZH',
                        'З': 'Z',
                        'И': 'I',
                        'Й': 'I',
                        'К': 'K',
                        'Л': 'L',
                        'М': 'M',
                        'Н': 'N',
                        'О': 'O',
                        'П': 'P',
                        'Р': 'R',
                        'С': 'S',
                        'Т': 'T',
                        'У': 'U',
                        'Ф': 'F',
                        'Х': 'KH',
                        'Ц': 'TS',
                        'Ч': 'CH',
                        'Ш': 'SH',
                        'Щ': 'SHCH',
                        'Ы': 'Y',
                        'Ъ': 'IE',
                        'Э': 'E',
                        'Ю': 'IU',
                        'Я': 'IA'}
cols = dummy_stations.columns.to_list()
dict_for_stations = {}
for station in cols:
    station_temp = station.upper()
    new = []
    for i in range(len(station_temp)):
        if station_temp[i] in transliteration_dict:
            new.append(transliteration_dict[station_temp[i]].lower())
        elif i == ' ' or i == '-':
            new.append('_')
    dict_for_stations[station] = ''.join(new)
dummy_stations.rename(columns=dict_for_stations, inplace=True)
df = pd.concat([df, dummy_stations], axis=1)
df.drop(columns=['stations'], inplace=True)
df.to_csv('./data.csv', index=False)  